In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

np.random.seed(601)

import sklearn
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.preprocessing import LabelEncoder

# 2. 데이터 전처리
## Data Cleansing & Pre-Processing

In [2]:
data = pd.read_csv('data/201901-202003.csv')

In [3]:
data.head()
data.tail()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3


,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
24697787,202003,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,3,43300,4
24697788,202003,충북,충주시,휴양콘도 운영업,충북,충주시,40s,1,3,3,35000,3
24697789,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,4,188000,6
24697790,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,2,4,4,99000,6
24697791,202003,충북,충주시,휴양콘도 운영업,충북,충주시,60s,1,5,3,194000,3


In [4]:
data = data.fillna('')

In [5]:
# CARD_SIDO_NM HOM_SIDO_NM 비교해서 같은지 다른지로만 판단 

idx_0 = data[(data.CARD_SIDO_NM == data.HOM_SIDO_NM) == True].index
data.loc[idx_0, 'HOM_SIDO_NM'] = 0
idx_1 = data[(data.CARD_SIDO_NM == data.HOM_SIDO_NM) == False].index
data.loc[idx_1, 'HOM_SIDO_NM'] = 1
data

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,1,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,1,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,1,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,1,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,1,강릉시,40s,1,4,3,1047300,3
...,...,...,...,...,...,...,...,...,...,...,...,...
24697787,202003,충북,충주시,휴양콘도 운영업,1,충주시,30s,1,2,3,43300,4
24697788,202003,충북,충주시,휴양콘도 운영업,1,충주시,40s,1,3,3,35000,3
24697789,202003,충북,충주시,휴양콘도 운영업,1,충주시,50s,1,4,4,188000,6
24697790,202003,충북,충주시,휴양콘도 운영업,1,충주시,50s,2,4,4,99000,6


In [6]:
data.drop(['CARD_CCG_NM','HOM_CCG_NM','CSTMR_CNT'], inplace=True, axis=1)
data

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,CNT
0,201901,강원,건강보조식품 소매업,1,20s,1,1,311200,4
1,201901,강원,건강보조식품 소매업,1,30s,1,2,1374500,8
2,201901,강원,건강보조식품 소매업,1,30s,2,2,818700,6
3,201901,강원,건강보조식품 소매업,1,40s,1,3,1717000,5
4,201901,강원,건강보조식품 소매업,1,40s,1,4,1047300,3
...,...,...,...,...,...,...,...,...,...
24697787,202003,충북,휴양콘도 운영업,1,30s,1,2,43300,4
24697788,202003,충북,휴양콘도 운영업,1,40s,1,3,35000,3
24697789,202003,충북,휴양콘도 운영업,1,50s,1,4,188000,6
24697790,202003,충북,휴양콘도 운영업,1,50s,2,4,99000,6


In [7]:
merge = pd.read_csv('data/submission.csv', index_col=0)
merge.drop(['AMT'], axis=1, inplace=True)

index = merge[merge['REG_YYMM'] == 202007].index
merge.drop(index, axis=0, inplace=True)

merge.shape
merge

(697, 3)

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM
id,,,
0,202004,강원,건강보조식품 소매업
1,202004,강원,골프장 운영업
2,202004,강원,과실 및 채소 소매업
3,202004,강원,관광 민예품 및 선물용품 소매업
4,202004,강원,그외 기타 분류안된 오락관련 서비스업
...,...,...,...
692,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업
693,202004,충북,한식 음식점업
694,202004,충북,호텔업


In [8]:
sub_col = []
for REG_YYMM in data['REG_YYMM'].unique():
    for CARD_SIDO_NM in merge['CARD_SIDO_NM'].unique():
        for STD_CLSS_CD in merge['STD_CLSS_NM'].unique():
            sub_col.append([REG_YYMM, CARD_SIDO_NM, STD_CLSS_CD])
sub_col = np.array(sub_col)
sub_col = pd.DataFrame(data=sub_col, columns=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'])
sub_col['AMT'] = 1
sub_col

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,강원,건강보조식품 소매업,1
1,201901,강원,골프장 운영업,1
2,201901,강원,과실 및 채소 소매업,1
3,201901,강원,관광 민예품 및 선물용품 소매업,1
4,201901,강원,그외 기타 분류안된 오락관련 서비스업,1
...,...,...,...,...
10450,202003,충북,피자 햄버거 샌드위치 및 유사 음식점업,1
10451,202003,충북,한식 음식점업,1
10452,202003,충북,호텔업,1
10453,202003,충북,화장품 및 방향제 소매업,1


In [9]:
df_dup = data[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']].drop_duplicates()
df_dup['AMT'] = 0
df_dup

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,강원,건강보조식품 소매업,0
13,201901,강원,골프장 운영업,0
31,201901,강원,과실 및 채소 소매업,0
484,201901,강원,관광 민예품 및 선물용품 소매업,0
508,201901,강원,그외 기타 스포츠시설 운영업,0
...,...,...,...,...
24664590,202003,충북,기타 대형 종합 소매업,0
24664639,202003,충북,기타 수상오락 서비스업,0
24677055,202003,충북,택시 운송업,0
24677781,202003,충북,그외 기타 스포츠시설 운영업,0


In [10]:
df_dup['REG_YYMM'] = df_dup['REG_YYMM'].astype(str)
df_dup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9438 entries, 0 to 24687181
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   REG_YYMM      9438 non-null   object
 1   CARD_SIDO_NM  9438 non-null   object
 2   STD_CLSS_NM   9438 non-null   object
 3   AMT           9438 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 368.7+ KB


In [11]:
df_merge = sub_col.merge(df_dup, left_on=['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM'], how='outer')
df_merge

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT_x,AMT_y
0,201901,강원,건강보조식품 소매업,1,0.0
1,201901,강원,골프장 운영업,1,0.0
2,201901,강원,과실 및 채소 소매업,1,0.0
3,201901,강원,관광 민예품 및 선물용품 소매업,1,0.0
4,201901,강원,그외 기타 분류안된 오락관련 서비스업,1,NaN
...,...,...,...,...,...
10450,202003,충북,피자 햄버거 샌드위치 및 유사 음식점업,1,0.0
10451,202003,충북,한식 음식점업,1,0.0
10452,202003,충북,호텔업,1,0.0
10453,202003,충북,화장품 및 방향제 소매업,1,0.0


In [12]:
null_idx = df_merge[df_merge['AMT_y'].isnull() == False].index
null_idx

df_merge.drop(null_idx, axis=0, inplace=True)
df_merge

Int64Index([    0,     1,     2,     3,     5,     6,     7,     8,     9,
               10,
            ...
            10445, 10446, 10447, 10448, 10449, 10450, 10451, 10452, 10453,
            10454],
           dtype='int64', length=9438)

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT_x,AMT_y
4,201901,강원,그외 기타 분류안된 오락관련 서비스업,1,NaN
30,201901,강원,정기 항공 운송업,1,NaN
45,201901,경기,그외 기타 분류안된 오락관련 서비스업,1,NaN
53,201901,경기,내항 여객 운송업,1,NaN
55,201901,경기,면세점,1,NaN
...,...,...,...,...,...
10428,202003,충북,면세점,1,NaN
10429,202003,충북,버스 운송업,1,NaN
10437,202003,충북,여행사업,1,NaN
10442,202003,충북,자동차 임대업,1,NaN


In [13]:
data.info()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24697792 entries, 0 to 24697791
Data columns (total 9 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   REG_YYMM      int64 
 1   CARD_SIDO_NM  object
 2   STD_CLSS_NM   object
 3   HOM_SIDO_NM   object
 4   AGE           object
 5   SEX_CTGO_CD   int64 
 6   FLC           int64 
 7   AMT           int64 
 8   CNT           int64 
dtypes: int64(5), object(4)
memory usage: 1.7+ GB


,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,CNT
0,201901,강원,건강보조식품 소매업,1,20s,1,1,311200,4
1,201901,강원,건강보조식품 소매업,1,30s,1,2,1374500,8
2,201901,강원,건강보조식품 소매업,1,30s,2,2,818700,6
3,201901,강원,건강보조식품 소매업,1,40s,1,3,1717000,5
4,201901,강원,건강보조식품 소매업,1,40s,1,4,1047300,3
...,...,...,...,...,...,...,...,...,...
24697787,202003,충북,휴양콘도 운영업,1,30s,1,2,43300,4
24697788,202003,충북,휴양콘도 운영업,1,40s,1,3,35000,3
24697789,202003,충북,휴양콘도 운영업,1,50s,1,4,188000,6
24697790,202003,충북,휴양콘도 운영업,1,50s,2,4,99000,6


In [13]:
df_merge['HOM_SIDO_NM'] = 0
df_merge['AGE'] = '00s'
df_merge['SEX_CTGO_CD'] = 0
df_merge['FLC'] = 0
df_merge['AMT'] = 0
df_merge['CNT'] = 0
df_merge.drop(['AMT_x','AMT_y'], axis=1, inplace=True)
df_merge

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,CNT
4,201901,강원,그외 기타 분류안된 오락관련 서비스업,0,00s,0,0,0,0
30,201901,강원,정기 항공 운송업,0,00s,0,0,0,0
45,201901,경기,그외 기타 분류안된 오락관련 서비스업,0,00s,0,0,0,0
53,201901,경기,내항 여객 운송업,0,00s,0,0,0,0
55,201901,경기,면세점,0,00s,0,0,0,0
...,...,...,...,...,...,...,...,...,...
10428,202003,충북,면세점,0,00s,0,0,0,0
10429,202003,충북,버스 운송업,0,00s,0,0,0,0
10437,202003,충북,여행사업,0,00s,0,0,0,0
10442,202003,충북,자동차 임대업,0,00s,0,0,0,0


In [14]:
concat_df = pd.concat([data, df_merge])
concat_df

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,CNT
0,201901,강원,건강보조식품 소매업,1,20s,1,1,311200,4
1,201901,강원,건강보조식품 소매업,1,30s,1,2,1374500,8
2,201901,강원,건강보조식품 소매업,1,30s,2,2,818700,6
3,201901,강원,건강보조식품 소매업,1,40s,1,3,1717000,5
4,201901,강원,건강보조식품 소매업,1,40s,1,4,1047300,3
...,...,...,...,...,...,...,...,...,...
10428,202003,충북,면세점,0,00s,0,0,0,0
10429,202003,충북,버스 운송업,0,00s,0,0,0,0
10437,202003,충북,여행사업,0,00s,0,0,0,0
10442,202003,충북,자동차 임대업,0,00s,0,0,0,0


In [15]:
data = concat_df
data

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,CNT
0,201901,강원,건강보조식품 소매업,1,20s,1,1,311200,4
1,201901,강원,건강보조식품 소매업,1,30s,1,2,1374500,8
2,201901,강원,건강보조식품 소매업,1,30s,2,2,818700,6
3,201901,강원,건강보조식품 소매업,1,40s,1,3,1717000,5
4,201901,강원,건강보조식품 소매업,1,40s,1,4,1047300,3
...,...,...,...,...,...,...,...,...,...
10428,202003,충북,면세점,0,00s,0,0,0,0
10429,202003,충북,버스 운송업,0,00s,0,0,0,0
10437,202003,충북,여행사업,0,00s,0,0,0,0
10442,202003,충북,자동차 임대업,0,00s,0,0,0,0


In [16]:
data['AGE'] = data['AGE'].str.get(0)

### 날짜 처리 YYYYMM -> YYYY + MM

In [17]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [18]:
# all
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
#data = data.drop(['REG_YYMM'], axis=1)

### 데이터 정제

In [19]:
# all

data.drop(['REG_YYMM'], inplace=True, axis=1)

df = data.copy()

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month'] # , 'STD_CLSS_CD'
df = df.groupby(columns).sum().reset_index(drop=False)

In [ ]:
df

In [ ]:
df.info()

In [20]:
df['CARD_SIDO_NM']=df['CARD_SIDO_NM'].astype(str)
#df['STD_CLSS_CD']=df['STD_CLSS_CD'].astype(str)
df['STD_CLSS_NM']=df['STD_CLSS_NM'].astype(str)

#df['REG_YYMM']=df['REG_YYMM'].astype(int)
df['HOM_SIDO_NM']=df['HOM_SIDO_NM'].astype(int)
df['AGE']=df['AGE'].astype(int)
df['SEX_CTGO_CD']=df['SEX_CTGO_CD'].astype(int)
df['FLC']=df['FLC'].astype(int)
df['year']=df['year'].astype(int)
df['month']=df['month'].astype(int)
df['AMT']=df['AMT'].astype(float)
df['CNT']=df['CNT'].astype(int)

### 인코딩

In [21]:
df.shape

(198618, 10)

In [22]:
df_num = pd.get_dummies(df)

In [23]:
df_num.shape

(198618, 66)

In [ ]:
# all
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])
    
df_num    

In [ ]:
colormap = plt.cm.RdBu
plt.figure(figsize=(14, 12))
sns.heatmap(df_num.astype(float).corr(), linewidths=0.1, vmax=1.0,
           square=True, cmap=colormap, linecolor='white', annot=True, annot_kws={"size": 16})

# 3. 탐색적 자료분석
## Exploratory Data Analysis

In [ ]:
# 입력하세요.


# 4. 변수 선택 및 모델 구축
## Feature Engineering & Initial Modeling

In [25]:
df_num.shape 
df_num 

(198618, 66)

,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,AMT,CNT,CARD_SIDO_NM_강원,CARD_SIDO_NM_경기,...,STD_CLSS_NM_중식 음식점업,STD_CLSS_NM_차량용 가스 충전업,STD_CLSS_NM_차량용 주유소 운영업,STD_CLSS_NM_체인화 편의점,STD_CLSS_NM_택시 운송업,STD_CLSS_NM_피자 햄버거 샌드위치 및 유사 음식점업,STD_CLSS_NM_한식 음식점업,STD_CLSS_NM_호텔업,STD_CLSS_NM_화장품 및 방향제 소매업,STD_CLSS_NM_휴양콘도 운영업
0,1,2,1,1,2019,1,311200.0,4,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,1,1,2019,2,605000.0,3,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,1,1,2019,6,139000.0,3,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2,1,1,2019,7,21000.0,3,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,2,1,1,2019,8,95000.0,12,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198613,1,7,2,5,2019,5,329800.0,7,0,0,...,0,0,0,0,0,0,0,0,0,1
198614,1,7,2,5,2019,6,333800.0,5,0,0,...,0,0,0,0,0,0,0,0,0,1
198615,1,7,2,5,2019,10,2670000.0,21,0,0,...,0,0,0,0,0,0,0,0,0,1
198616,1,7,2,5,2019,12,247800.0,3,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
# 201908 ~ 202002 -> train
# 202003          -> test

train = df_num[df_num['REG_YYMM'] != 202003]
test = df_num[df_num['REG_YYMM'] == 202003]

In [ ]:
train.shape
test.shape

In [24]:
feature = df_num.drop([ 'AMT', 'CNT'], axis=1)
target = np.log1p(df_num['AMT'])

In [25]:
df.drop(['AMT', 'CNT'], axis=1, inplace=True)

### feature, target 설정

In [ ]:
train_feature = train.drop([ 'REG_YYMM','AMT', 'CNT'], axis=1)
train_target = np.log1p(train['AMT'])

test_feature = test.drop([ 'REG_YYMM','AMT', 'CNT'], axis=1)
test_target = np.log1p(test['AMT'])

# 5. 모델 학습 및 검증
## Model Tuning & Evaluation

In [49]:
model = ExtraTreesRegressor(n_jobs=-1, random_state=0)

In [50]:
model.fit(feature, target)

ExtraTreesRegressor(n_jobs=-1, random_state=0)

In [126]:
from sklearn.linear_model import LassoCV, Lasso
model = Lasso(alpha=0.05)
model.fit(feature, target)

Lasso(alpha=0.05)

In [1]:
from lightgbm import LGBMRegressor
model = LGBMRegressor()
model.fit(feature, target)

ModuleNotFoundError: No module named 'lightgbm'

In [129]:
feature.shape

(198618, 64)

# 6. 결과 및 결언
## Conclusion & Discussion

In [29]:
# 예측 템플릿 만들기
HOM_SIDO_NMs  = df['HOM_SIDO_NM'].unique()
AGEs          = df['AGE'].unique()
SEX_CTGO_CDs  = df['SEX_CTGO_CD'].unique()
FLCs          = df['FLC'].unique()
years         = [2020]
months        = [4, 7]
CARD_SIDO_NMs = df['CARD_SIDO_NM'].unique()
#STD_CLSS_CDs  = df_num['STD_CLSS_CD'].unique()
STD_CLSS_NMs  = df['STD_CLSS_NM'].unique()

In [30]:
temp1 = []

for HOM_SIDO_NM in HOM_SIDO_NMs:
    for AGE in AGEs:
        for SEX_CTGO_CD in SEX_CTGO_CDs:
            for FLC in FLCs:
                for year in years:
                    for month in months:
                        for CARD_SIDO_NM in CARD_SIDO_NMs:
                            for STD_CLSS_NM in STD_CLSS_NMs:
                                temp1.append([HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month, CARD_SIDO_NM, STD_CLSS_NM])
temp1 = np.array(temp1)
temp1

array([['1', '2', '1', ..., '4', '강원', '건강보조식품 소매업'],
       ['1', '2', '1', ..., '4', '강원', '골프장 운영업'],
       ['1', '2', '1', ..., '4', '강원', '과실 및 채소 소매업'],
       ...,
       ['0', '0', '0', ..., '7', '충북', '호텔업'],
       ['0', '0', '0', ..., '7', '충북', '화장품 및 방향제 소매업'],
       ['0', '0', '0', ..., '7', '충북', '휴양콘도 운영업']], dtype='<U21')

In [99]:
temp1.shape

(401472, 8)

In [131]:
temp = pd.DataFrame(data=temp1, columns=['HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD','FLC', 'year', 'month', 'CARD_SIDO_NM', 'STD_CLSS_NM'],)

In [132]:
temp

,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CARD_SIDO_NM,STD_CLSS_NM
0,1,2,1,1,2020,4,강원,건강보조식품 소매업
1,1,2,1,1,2020,4,강원,골프장 운영업
2,1,2,1,1,2020,4,강원,과실 및 채소 소매업
3,1,2,1,1,2020,4,강원,관광 민예품 및 선물용품 소매업
4,1,2,1,1,2020,4,강원,그외 기타 분류안된 오락관련 서비스업
...,...,...,...,...,...,...,...,...
401467,0,0,0,0,2020,7,충북,피자 햄버거 샌드위치 및 유사 음식점업
401468,0,0,0,0,2020,7,충북,한식 음식점업
401469,0,0,0,0,2020,7,충북,호텔업
401470,0,0,0,0,2020,7,충북,화장품 및 방향제 소매업


In [133]:
dum = pd.get_dummies(temp[['CARD_SIDO_NM', 'STD_CLSS_NM']])

In [134]:
other = temp[['HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD','FLC', 'year', 'month']]

In [135]:
pd.concat([other, dum], axis=1)

,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CARD_SIDO_NM_강원,CARD_SIDO_NM_경기,CARD_SIDO_NM_경남,CARD_SIDO_NM_경북,...,STD_CLSS_NM_중식 음식점업,STD_CLSS_NM_차량용 가스 충전업,STD_CLSS_NM_차량용 주유소 운영업,STD_CLSS_NM_체인화 편의점,STD_CLSS_NM_택시 운송업,STD_CLSS_NM_피자 햄버거 샌드위치 및 유사 음식점업,STD_CLSS_NM_한식 음식점업,STD_CLSS_NM_호텔업,STD_CLSS_NM_화장품 및 방향제 소매업,STD_CLSS_NM_휴양콘도 운영업
0,1,2,1,1,2020,4,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,1,1,2020,4,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,1,1,2020,4,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2,1,1,2020,4,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,2,1,1,2020,4,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401467,0,0,0,0,2020,7,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
401468,0,0,0,0,2020,7,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
401469,0,0,0,0,2020,7,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
401470,0,0,0,0,2020,7,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [136]:
temp = pd.concat([other, dum], axis=1)

In [137]:
temp.shape

(401472, 64)

In [138]:
# 예측
pred = model.predict(temp)
pred = np.expm1(pred)
pred.shape
pred

(401472,)

array([6098740.72935012, 6098740.72935012, 6098740.72935012, ...,
         18068.78180596,   18068.78180596,   18068.78180596])

In [139]:
temp['AMT'] = np.round(pred, 0)

In [140]:
temp['REG_YYMM'] = temp['year'] +'0'+ temp['month']

In [141]:
temp

,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CARD_SIDO_NM_강원,CARD_SIDO_NM_경기,CARD_SIDO_NM_경남,CARD_SIDO_NM_경북,...,STD_CLSS_NM_차량용 주유소 운영업,STD_CLSS_NM_체인화 편의점,STD_CLSS_NM_택시 운송업,STD_CLSS_NM_피자 햄버거 샌드위치 및 유사 음식점업,STD_CLSS_NM_한식 음식점업,STD_CLSS_NM_호텔업,STD_CLSS_NM_화장품 및 방향제 소매업,STD_CLSS_NM_휴양콘도 운영업,AMT,REG_YYMM
0,1,2,1,1,2020,4,1,0,0,0,...,0,0,0,0,0,0,0,0,6098741.0,202004
1,1,2,1,1,2020,4,1,0,0,0,...,0,0,0,0,0,0,0,0,6098741.0,202004
2,1,2,1,1,2020,4,1,0,0,0,...,0,0,0,0,0,0,0,0,6098741.0,202004
3,1,2,1,1,2020,4,1,0,0,0,...,0,0,0,0,0,0,0,0,3704301.0,202004
4,1,2,1,1,2020,4,1,0,0,0,...,0,0,0,0,0,0,0,0,6098741.0,202004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401467,0,0,0,0,2020,7,0,0,0,0,...,0,0,0,1,0,0,0,0,18069.0,202007
401468,0,0,0,0,2020,7,0,0,0,0,...,0,0,0,0,1,0,0,0,129051.0,202007
401469,0,0,0,0,2020,7,0,0,0,0,...,0,0,0,0,0,1,0,0,18069.0,202007
401470,0,0,0,0,2020,7,0,0,0,0,...,0,0,0,0,0,0,1,0,18069.0,202007


In [60]:
temp.columns
temp.columns.str.find('CARD_SIDO_NM')
temp.columns.str.find('STD_CLSS_NM')

Index(['HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month',
       'CARD_SIDO_NM_강원', 'CARD_SIDO_NM_경기', 'CARD_SIDO_NM_경남',
       'CARD_SIDO_NM_경북', 'CARD_SIDO_NM_광주', 'CARD_SIDO_NM_대구',
       'CARD_SIDO_NM_대전', 'CARD_SIDO_NM_부산', 'CARD_SIDO_NM_서울',
       'CARD_SIDO_NM_세종', 'CARD_SIDO_NM_울산', 'CARD_SIDO_NM_인천',
       'CARD_SIDO_NM_전남', 'CARD_SIDO_NM_전북', 'CARD_SIDO_NM_제주',
       'CARD_SIDO_NM_충남', 'CARD_SIDO_NM_충북', 'STD_CLSS_NM_건강보조식품 소매업',
       'STD_CLSS_NM_골프장 운영업', 'STD_CLSS_NM_과실 및 채소 소매업',
       'STD_CLSS_NM_관광 민예품 및 선물용품 소매업', 'STD_CLSS_NM_그외 기타 분류안된 오락관련 서비스업',
       'STD_CLSS_NM_그외 기타 스포츠시설 운영업', 'STD_CLSS_NM_그외 기타 종합 소매업',
       'STD_CLSS_NM_기타 대형 종합 소매업', 'STD_CLSS_NM_기타 수상오락 서비스업',
       'STD_CLSS_NM_기타 외국식 음식점업', 'STD_CLSS_NM_기타 주점업',
       'STD_CLSS_NM_기타음식료품위주종합소매업', 'STD_CLSS_NM_내항 여객 운송업',
       'STD_CLSS_NM_마사지업', 'STD_CLSS_NM_면세점', 'STD_CLSS_NM_버스 운송업',
       'STD_CLSS_NM_비알콜 음료점업', 'STD_CLSS_NM_빵 및 과자류 소매업',
       'STD_CLSS_NM_서양식 음식점업', 'STD_

Int64Index([-1, -1, -1, -1, -1, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
             0,  0,  0,  0,  0,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
            -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
            -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
           dtype='int64')

Int64Index([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
            -1, -1, -1, -1, -1, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
             0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
             0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1, -1],
           dtype='int64')

In [98]:
temp

,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CARD_SIDO_NM_강원,CARD_SIDO_NM_경기,CARD_SIDO_NM_경남,CARD_SIDO_NM_경북,...,STD_CLSS_NM_중식 음식점업,STD_CLSS_NM_차량용 가스 충전업,STD_CLSS_NM_차량용 주유소 운영업,STD_CLSS_NM_체인화 편의점,STD_CLSS_NM_택시 운송업,STD_CLSS_NM_피자 햄버거 샌드위치 및 유사 음식점업,STD_CLSS_NM_한식 음식점업,STD_CLSS_NM_호텔업,STD_CLSS_NM_화장품 및 방향제 소매업,STD_CLSS_NM_휴양콘도 운영업
0,1,2,1,1,2020,4,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,1,1,2020,4,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,1,1,2020,4,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2,1,1,2020,4,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,2,1,1,2020,4,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401467,0,0,0,0,2020,7,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
401468,0,0,0,0,2020,7,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
401469,0,0,0,0,2020,7,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
401470,0,0,0,0,2020,7,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [142]:
temp2 = temp[['REG_YYMM','HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC','AMT']]

In [143]:
temp2['CARD_SIDO_NM'] = temp[['CARD_SIDO_NM_강원', 'CARD_SIDO_NM_경기', 'CARD_SIDO_NM_경남',
       'CARD_SIDO_NM_경북', 'CARD_SIDO_NM_광주', 'CARD_SIDO_NM_대구',
       'CARD_SIDO_NM_대전', 'CARD_SIDO_NM_부산', 'CARD_SIDO_NM_서울',
       'CARD_SIDO_NM_세종', 'CARD_SIDO_NM_울산', 'CARD_SIDO_NM_인천',
       'CARD_SIDO_NM_전남', 'CARD_SIDO_NM_전북', 'CARD_SIDO_NM_제주',
       'CARD_SIDO_NM_충남', 'CARD_SIDO_NM_충북']].idxmax(axis=1).str.replace(pat='CARD_SIDO_NM_', repl='', regex=False)

In [144]:
temp2['STD_CLSS_NM'] = temp[['STD_CLSS_NM_건강보조식품 소매업',
       'STD_CLSS_NM_골프장 운영업', 'STD_CLSS_NM_과실 및 채소 소매업',
       'STD_CLSS_NM_관광 민예품 및 선물용품 소매업', 'STD_CLSS_NM_그외 기타 분류안된 오락관련 서비스업',
       'STD_CLSS_NM_그외 기타 스포츠시설 운영업', 'STD_CLSS_NM_그외 기타 종합 소매업',
       'STD_CLSS_NM_기타 대형 종합 소매업', 'STD_CLSS_NM_기타 수상오락 서비스업',
       'STD_CLSS_NM_기타 외국식 음식점업', 'STD_CLSS_NM_기타 주점업',
       'STD_CLSS_NM_기타음식료품위주종합소매업', 'STD_CLSS_NM_내항 여객 운송업',
       'STD_CLSS_NM_마사지업', 'STD_CLSS_NM_면세점', 'STD_CLSS_NM_버스 운송업',
       'STD_CLSS_NM_비알콜 음료점업', 'STD_CLSS_NM_빵 및 과자류 소매업',
       'STD_CLSS_NM_서양식 음식점업', 'STD_CLSS_NM_수산물 소매업', 'STD_CLSS_NM_슈퍼마켓',
       'STD_CLSS_NM_스포츠 및 레크레이션 용품 임대업', 'STD_CLSS_NM_여관업', 'STD_CLSS_NM_여행사업',
       'STD_CLSS_NM_욕탕업', 'STD_CLSS_NM_육류 소매업', 'STD_CLSS_NM_일반유흥 주점업',
       'STD_CLSS_NM_일식 음식점업', 'STD_CLSS_NM_자동차 임대업', 'STD_CLSS_NM_전시 및 행사 대행업',
       'STD_CLSS_NM_정기 항공 운송업', 'STD_CLSS_NM_중식 음식점업',
       'STD_CLSS_NM_차량용 가스 충전업', 'STD_CLSS_NM_차량용 주유소 운영업',
       'STD_CLSS_NM_체인화 편의점', 'STD_CLSS_NM_택시 운송업',
       'STD_CLSS_NM_피자 햄버거 샌드위치 및 유사 음식점업', 'STD_CLSS_NM_한식 음식점업',
       'STD_CLSS_NM_호텔업', 'STD_CLSS_NM_화장품 및 방향제 소매업', 'STD_CLSS_NM_휴양콘도 운영업']].idxmax(axis=1).str.replace(pat='STD_CLSS_NM_', repl='', regex=False)

In [145]:
temp2

,REG_YYMM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,CARD_SIDO_NM,STD_CLSS_NM
0,202004,1,2,1,1,6098741.0,강원,건강보조식품 소매업
1,202004,1,2,1,1,6098741.0,강원,골프장 운영업
2,202004,1,2,1,1,6098741.0,강원,과실 및 채소 소매업
3,202004,1,2,1,1,3704301.0,강원,관광 민예품 및 선물용품 소매업
4,202004,1,2,1,1,6098741.0,강원,그외 기타 분류안된 오락관련 서비스업
...,...,...,...,...,...,...,...,...
401467,202007,0,0,0,0,18069.0,충북,피자 햄버거 샌드위치 및 유사 음식점업
401468,202007,0,0,0,0,129051.0,충북,한식 음식점업
401469,202007,0,0,0,0,18069.0,충북,호텔업
401470,202007,0,0,0,0,18069.0,충북,화장품 및 방향제 소매업


In [146]:
#temp['AMT'] = np.round(pred, 0)
#temp['REG_YYMM'] = temp['year']*100 + temp['month']
#temp['REG_YYMM'] = temp['year'] + temp['month']
temp3 = temp2[[ 'REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp3

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,6098741.0
1,202004,강원,골프장 운영업,6098741.0
2,202004,강원,과실 및 채소 소매업,6098741.0
3,202004,강원,관광 민예품 및 선물용품 소매업,3704301.0
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,6098741.0
...,...,...,...,...
401467,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,18069.0
401468,202007,충북,한식 음식점업,129051.0
401469,202007,충북,호텔업,18069.0
401470,202007,충북,화장품 및 방향제 소매업,18069.0


In [147]:
temp4 = temp3.groupby([  'REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)
temp4

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,1.943652e+09
1,202004,강원,골프장 운영업,1.943652e+09
2,202004,강원,과실 및 채소 소매업,1.943652e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.180551e+09
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.943652e+09
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.950846e+09
1390,202007,충북,한식 음식점업,1.393271e+10
1391,202007,충북,호텔업,1.950846e+09
1392,202007,충북,화장품 및 방향제 소매업,1.950846e+09


In [ ]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [90]:
temp

0       202004
1       202004
2       202004
3       202004
4       202004
         ...  
1389     20207
1390     20207
1391     20207
1392     20207
1393     20207
Name: REG_YYMM, Length: 1394, dtype: object

In [148]:
temp4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1394 entries, 0 to 1393
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   REG_YYMM      1394 non-null   object 
 1   CARD_SIDO_NM  1394 non-null   object 
 2   STD_CLSS_NM   1394 non-null   object 
 3   AMT           1394 non-null   float64
dtypes: float64(1), object(3)
memory usage: 43.7+ KB


In [149]:
temp4['REG_YYMM'] = temp4['REG_YYMM'].astype('int64')

In [150]:
# 제출 파일 만들기
submission = pd.read_csv('data/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp4, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('20200727_2.csv', encoding='utf-8-sig')
submission.head(30)

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,1.943652e+09
1,202004,강원,골프장 운영업,1.943652e+09
2,202004,강원,과실 및 채소 소매업,1.943652e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.180551e+09
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.943652e+09
5,202004,강원,그외 기타 스포츠시설 운영업,1.943652e+09
6,202004,강원,그외 기타 종합 소매업,1.943652e+09
7,202004,강원,기타 대형 종합 소매업,6.552274e+09
8,202004,강원,기타 수상오락 서비스업,1.943652e+09
